In [1]:
import numpy as np
import torch

In [2]:
t = torch.tensor([[1, 2], [3, 4]])
t

tensor([[1, 2],
        [3, 4]])

## TensorとArrayの型を比較

In [3]:
num1 = np.zeros((3, 4))
ten1 =torch.zeros((2, 3))
print(f"num1 : {num1.shape} / {num1.dtype}") #arrayは実数はfloat64。整数がint32
print(f"ten1 : {ten1.shape} / {ten1.dtype}") #tensorは実数はfloat32。整数はint64
print(type(ten1))

num1 : (3, 4) / float64
ten1 : torch.Size([2, 3]) / torch.float32
<class 'torch.Tensor'>


## Numpy Array とTensorの変換

In [4]:
before = torch.zeros(3)
print(f"変換前：{before}")

after = before.numpy() #TensorからArrayへ変換 → .numpy メソッド！！
print(f"変換後：{after}")

data = torch.from_numpy(after) #ArrayからTensorへ変換 → torch.from_numpy メソッド！！
print(f"再変換後：{data}")

変換前：tensor([0., 0., 0.])
変換後：[0. 0. 0.]
再変換後：tensor([0., 0., 0.])


In [5]:
#この時、メモリが共有されていることに注意する！！
before.add_(1)
print(f"変換前：{before}")
print(f"変換後：{after}")
print(f"再変換後：{data}")

変換前：tensor([1., 1., 1.])
変換後：[1. 1. 1.]
再変換後：tensor([1., 1., 1.])


## 自動微分

In [6]:
 x = torch.tensor(1.0, requires_grad = True) # 1.0はx = 1の時の出力を表す。requires_gradは自動微分機能に関する記述。
a, b = 3, 5
y = a*x + b  # y = 3x + 5  これ、順伝播。

print(y)

tensor(8., grad_fn=<AddBackward0>)


In [7]:
y.backward()  #計算後のTensor名.backward() で微分が行われる。これ、逆伝播。
# Tensorで宣言されていない変数は微分できないので注意。

In [8]:
print(x.grad)  #x.gradはy = 3x + 5 をxで微分した値＝３。勾配の計算。

tensor(3.)


In [9]:
y = a*x + b
y.backward()
print(x.grad) #backward()が実行される度に、Tensor名.gradは加算を繰り返してしまう。

tensor(6.)


In [10]:
# 例題演習
v = torch.tensor(1.0, requires_grad = True)
w = torch.tensor(1.0, requires_grad = True)
out = 4*v + 6 *w + 1

out.backward()
print(v.grad)
print(w.grad)

tensor(4.)
tensor(6.)


## transposeとreshape : 次元の調整に使う

In [11]:
x = torch.rand((4, 3))
print(x)
print(x.transpose(0, 1)) #どっちでも転置
print(torch.t(x))              #どっちでも転置

tensor([[0.5239, 0.4946, 0.4096],
        [0.2965, 0.0912, 0.7677],
        [0.9280, 0.3570, 0.7567],
        [0.5698, 0.2118, 0.7391]])
tensor([[0.5239, 0.2965, 0.9280, 0.5698],
        [0.4946, 0.0912, 0.3570, 0.2118],
        [0.4096, 0.7677, 0.7567, 0.7391]])
tensor([[0.5239, 0.2965, 0.9280, 0.5698],
        [0.4946, 0.0912, 0.3570, 0.2118],
        [0.4096, 0.7677, 0.7567, 0.7391]])


In [12]:
x = torch.ones(16)
print(x)
print(x.reshape(2, 8))
print(x.reshape(4, -1))

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


#### Zero_()関数 ：行列を全てゼロに書き換える

In [13]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(a)
a.zero_()  #"_"はインプレース（破壊的メソッド）を表す。変数の値を変更してしまうので注意が必要。
print(a)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[0, 0, 0],
        [0, 0, 0]])


## 全結合層と活性化関数

#### 全結合層

In [14]:
import torch
from torch import nn #全結合層のため
from torch.nn import functional as F #活性化関数のため

In [15]:
# 全結合層 fc を定義して、Tensor[1, 2, 3, 4] を伝播させる。
# 4次元ベクトルを2次元に変換する全結合層をクラスとして定義
fc = nn.Linear(4, 2)      # 入力が4, 出力が2の全結合層。４つの数値から２つの数値を計算してくれる「いい感じの何か」.
x = torch.Tensor([1, 2, 3, 4])

# fc の__call__()メソッドを呼ぶことでx を伝播できる。
x = fc(x)
print(x)

tensor([-1.4044,  0.2114], grad_fn=<AddBackward0>)


[説明]
・tensor([x, y]) のx, yは[1,2, 3, 4] をfc_layerの初期パラメータで変換した値。
 4次元tensorが２次元に変換されていることがわかる。
・fc をインスタンスとして宣言し、__call__() で伝播させる。

#### 活性化関数：relu関数

In [16]:
# 次に、活性化関数を実装する。その前に、relu() 関数を理解。
x = torch.Tensor([-1.0, -0.5, 0.5, 1.0])
x = F. relu(x)
print(x)  #０以下の値が「0.0000」となっている。

tensor([0.0000, 0.0000, 0.5000, 1.0000])


In [17]:
### これで順伝播は終わり！
# x とfc の用意。
x = torch.Tensor([-2, -1, 1, 2])
fc = nn.Linear(4, 2)

# 全結合層を通じたx を表示。
print(fc(x))

# さらに、relu 関数を適用したx を表示。
print(F.relu(x))

tensor([-0.0744, -0.3606], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 2.])


## 損失関数と最適化関数

In [18]:
from torch import optim
from torchvision.models import vgg11

#### まずは、損失関数をクラスとして用意

In [19]:
criterion = nn.MSELoss()

# 下記２つのTensorのMSE（平均二乗誤差）を求める。
x = torch.Tensor([0, 1, 2])
y = torch.Tensor([1, -1, 0])

loss = criterion(x, y)

print(loss)  # たったこれだけで　MSEが計算できてしまう！すごい。

tensor(3.)


#### 次に、最適化関数を実装する →今回はAdamを使用

In [20]:
# モデルを用意する
model = vgg11()

# 用意したモデルのパラメーターを与えて、クラスを定義する。
optimizer = optim.Adam(model.parameters())  #Adamには、パラメーターが必要！
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


#### 損失関数MSEを宣言して、損失を計算してみる。

criterion = nn.MSELoss()

x = torch.Tensor([1, 1, 1, 1])
y = torch.Tensor([0, 2, 4, 6])

loss = criterion(x, y)

print(loss)

## モデルの定義

#### 初期化関数"__init__()"で、モデルに使うレイヤーや関数を準備し、forward()関数でモデルを組み立てる。

In [21]:
class mlp_net(nn.Module):      #nn.Moduleが基底クラス、mlp_netが派生クラス。
    def __init__(self):
        super(). __init__ ()         # 基底クラスのデータ属性を初期化。下記のself.fc1, self.fc2.....たちを外から代入するデータ属性として設定。
        
        #全結合層を２つ
        self.fc1 = nn.Linear(3, 5) #入力層が3次元、中間層が５次元、出力層が２次元
        self.fc2 = nn.Linear(5, 2)
        
        #損失関数と最適化関数
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters())  #Adamのパラメーター設定は、nn.Moduleが勝手にやってくれるので、心配なし！
    
    def forward(self, x) :             # forward()メソッドの定義。外部引数はx. これが順伝播。
        x = self.fc1(x)
        print('[fc1を通過]\n', x)    #中間層の出力を表示
        
        #  活性化関数
        x = F.relu(x)
        print('[relu()を通過]\n', x)  # relu()の適用結果を表示
        
        x = self.fc2(x)
        return x

#### 作成したモデルに、tensor[0, 1, 2]を伝播させてみる。

In [22]:
# モデルを宣言
model = mlp_net()

x = torch.Tensor([0, 1, 2])

# xを伝播させる
output = model(x)

print('[モデルの出力]\n', output)

[fc1を通過]
 tensor([ 0.0548,  1.4792, -1.5062, -0.5882, -1.1932], grad_fn=<AddBackward0>)
[relu()を通過]
 tensor([0.0548, 1.4792, 0.0000, 0.0000, 0.0000], grad_fn=<ReluBackward0>)
[モデルの出力]
 tensor([0.1737, 0.6141], grad_fn=<AddBackward0>)


[例題]


In [23]:
class mlp_net(nn.Module):      #nn.Moduleが基底クラス（使用必須）、mlp_netが派生クラス。
    def __init__(self):
        super(). __init__ ()         # 基底クラスのデータ属性を初期化。下記のself.fc1, self.fc2.....たちを外から代入するデータ属性として設定。
        
        #全結合層を２つ
        self.fc1 = nn.Linear(4, 3) #入力層が3次元、中間層が５次元、出力層が２次元
        self.fc2 = nn.Linear(3, 2)
        
        #損失関数と最適化関数
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters())  #Adamのパラメーター設定は、nn.Moduleが勝手にやってくれるので、心配なし！
    
    def forward(self, x) :             # forward()メソッドの定義。外部引数はx. これが順伝播。
        x = self.fc1(x)
        print('[fc1を通過]\n', x)    #中間層の出力を表示
        
        #  活性化関数
        x = F.relu(x)
        print('[relu()を通過]\n', x)  # relu()の適用結果を表示
        
        x = self.fc2(x)
        return x

In [24]:
mlp_net()

mlp_net(
  (fc1): Linear(in_features=4, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=2, bias=True)
  (criterion): MSELoss()
)

## MNISTデータセット

In [25]:

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor()) 
 #学習用データセット
train_loader = utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)  
 # ミニバッチごとにデータを纏める(学習時にはshuffle=True)\n",
 # DataLoder : ミニバッチを指定したら、勝手にデータを分けてくれる便利なもの。

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor()) 
 # 検証用データセット
test_loader = utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2) 
 # ミニバッチごとにデータを纏める(学習時にはshuffle=False)
 

'\ntrainset = datasets.MNIST(root=\'./data\', train=True, download=True, transform=transforms.ToTensor()) \n #学習用データセット\ntrain_loader = utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)  \n # ミニバッチごとにデータを纏める(学習時にはshuffle=True)\n",\n\ntestset = datasets.MNIST(root=\'./data\', train=False, download=True, transform=transforms.ToTensor()) \n # 検証用データセット\ntest_loader = utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2) \n # ミニバッチごとにデータを纏める(学習時にはshuffle=False)\n '

## モデルの定義

In [ ]:
class mlp_net(nn.Module):      #nn.Moduleが基底クラス、mlp_netが派生クラス。
    def __init__(self):
        super(). __init__ ()         # 基底クラスのデータ属性を初期化。下記のself.fc1, self.fc2.....たちを外から代入するデータ属性として設定。
        
        #全結合層を２つ
        self.fc1 = nn.Linear(784, 512) #入力層が784次元、中間層が５12次元、出力層が10次元
        self.fc2 = nn.Linear(512, 10)
        
        #損失関数と最適化関数
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters())  #Adamのパラメーター設定は、nn.Moduleが勝手にやってくれるので、心配なし！
    
    def forward(self, x) :             # forward()メソッドの定義。外部引数はx. これが順伝播。
        x = self.fc1(x)
        #  活性化関数
        x = F.relu(x)        
        x = self.fc2(x)
        return x

## Train関数の実装

### 注意点
①　２次元の画像であるテンソルを、１次元のテンソルに変換する\
→ reshape()を使う！！\
②　損失を求めるために正解データをone-hotベクトル化する

In [ ]:
def train(model, train_loder):
    
    model.train() # 今は学習中であることを明治するコード！！
    
    # ミニバッチごとにループさせる,train_loaderの中身を出し切ったら1エポックとなる
    for batch_imgs, batch_labels in train_loader:
        batch_imgs = batch_imgs.reshape(-1, 28*28*1)  # 画像データを1次元に変換
        labels = torch.eye(10)[batch_labels]  # 正解ラベルをone-hotベクトルへ変換

        outputs = model(batch_imgs)  # 順伝播
        model.optimizer.zero_grad()  # 勾配を初期化！！（前回のループ時の勾配を削除。backwardよりも前で宣言する必要あり）
        loss = model.criterion(outputs, labels)  # outputs と labelsの間の損失を計算
        loss.backward()  # 逆伝播で勾配を計算
        model.optimizer.step()  # 最適化（.step とすると最適化される）
    return

    

## 色々な活性化関数

In [26]:
import torch
from torch.nn import functional as F

x = torch.tensor([-2.0, 1.0, 4.0, 0.0])

print('relu :', F.relu(x))   # relu関数

print('softmax:', F.softmax(x, dim = 0)) 
　"""softmax関数 : 入力ベクトルの要素が０以上で、和が１になるようにスケール処理。：つまり、確率値に直すということ。
      １次元のベクトルのsoftmax関数を適用する場合は、引数dimに0を指定する。"""
    
print('sigmoid:', torch.sigmoid(x))  #sigmoid関数 : 入力値を0から1に収まるように処理する。

SyntaxError: invalid character in identifier (<ipython-input-26-4545548c67fc>, line 9)

## 色々な損失関数

### ①MSELoss関数

・２つのベクトルの要素ごとの差の２上から誤差を算出。\
・インスタンス宣言時に、引数"reduction = mean"で要素ごとの平均、"reduction = sum"で合計を損失値として出力。\
・"reduction = none"を指定すると、平均や合計処理をする前の、要素ごとの誤差をテンソルとして出力する。

### ②CrossEntropyLoss関数

・MSが「全要素の誤差を求めていたのに対し、「正解ラベルと対応する出力との差のみ」を計算。\
・しかし、正しくこれを用いるには、モデルの出力をsoftmax関数などで確率にする必要がある。\
・なぜならば、正解ラベルに対するモデルの出力が「１」の時に損失が０となってしまうからだ。

In [27]:
from torch import nn

# バッチサイズ=1 と見立てたデータを用意
x = torch.tensor([[0.2, 0.5, 0.3]])  #MSELossの場合は、入力の合計値が1になるように準備する必要がある。
mse_label = torch.tensor([[0, 1, 0]])  #crossentropyの場合は、上記のような制約はない。
cel_label = torch .tensor([1])

print('MSE:', nn.MSELoss(reduction = 'mean')(x, mse_label))
print('CrossEntropy:', nn.CrossEntropyLoss()(x, cel_label))

MSE: tensor(0.1267)
CrossEntropy: tensor(0.9398)


# 12/24 Pytorchチュートリアル

In [1]:
import torch

In [32]:
# long型（int型）の数値0で初期化された行列を生成
x = torch.zeros(5, 3, dtype = torch.long)
x

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [33]:
# 直接、数値を指定して行列を生成。（pythonのリスト型による作成）
x = torch.tensor([5.5, 3])
x

tensor([5.5000, 3.0000])

In [38]:
# すでにあるtensorを元に、新しくtensorを生成
x = x.new_ones(5, 3, dtype = torch.double)  # new_ * methods
print(x)

x = torch.randn_like(x, dtype=torch.float)  #_like で形状（５行３列）を継承できる。
    # randn : 平均が０、分散が１の乱数。
print(x)
print(x.size())

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.5100, -0.6161,  0.8510],
        [-0.0251, -0.2915, -0.1122],
        [-0.5384, -0.0136,  0.2812],
        [-0.2740, -1.0454, -0.8488],
        [ 0.9613, -0.9050, -0.4126]])
torch.Size([5, 3])


#### 🌟 torch.doubleはtorch.float64と同じ意味で、64bitの浮動小数点型を表す
尚、torch.half = torch.float16, torch.float = torch.float32
     torch.short = torch.int16, torch.int = torch.int32, torch.long = torch.int64 を表す。\
     そして、基本的に32bitのtorch.float か torch.int しか使わない。

#### （注意）Numpyの浮動小数点の数値が、デフォルトで64bitになっている点に注意。\
したがって、Numpyから変換したtensorは、
#### float() メソッド
を呼び出してデータ型の変換を行う必要がある！

## Tensorの演算

In [39]:
x

tensor([[ 0.5100, -0.6161,  0.8510],
        [-0.0251, -0.2915, -0.1122],
        [-0.5384, -0.0136,  0.2812],
        [-0.2740, -1.0454, -0.8488],
        [ 0.9613, -0.9050, -0.4126]])

In [40]:
y = torch.rand(5, 3)
y

tensor([[0.1686, 0.2283, 0.9823],
        [0.5158, 0.4037, 0.5889],
        [0.7457, 0.0056, 0.5473],
        [0.2205, 0.7648, 0.8726],
        [0.4483, 0.9911, 0.1036]])

In [41]:
# 足し算①
print(x + y)

tensor([[ 0.6785, -0.3879,  1.8333],
        [ 0.4907,  0.1122,  0.4767],
        [ 0.2073, -0.0079,  0.8284],
        [-0.0534, -0.2806,  0.0238],
        [ 1.4096,  0.0861, -0.3091]])


In [43]:
# 足し算②
print(torch.add(x, y))

tensor([[ 0.6785, -0.3879,  1.8333],
        [ 0.4907,  0.1122,  0.4767],
        [ 0.2073, -0.0079,  0.8284],
        [-0.0534, -0.2806,  0.0238],
        [ 1.4096,  0.0861, -0.3091]])


In [47]:
y.add_(x) #破壊的メソッド"_" により、x + y が y に代入された。←←これ、実行のたびに足し続けちゃうんだけど・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・
y

tensor([[ 2.2084, -2.2363,  4.3863],
        [ 0.4155, -0.7624,  0.1400],
        [-1.4080, -0.0487,  1.6719],
        [-0.8753, -3.4168, -2.5225],
        [ 4.2934, -2.6289, -1.5469]])

## スライシング

In [48]:
x = torch.randn(4, 4)
x

tensor([[-0.1167, -2.0825, -0.0997,  0.7579],
        [ 0.1968,  0.9784,  0.0714,  0.4933],
        [-1.5637, -0.4070, -0.3432, -2.0151],
        [-1.3312, -0.2478,  0.6550, -1.4580]])

In [49]:
y = x.view(16) #１６個のベクトル？    .viewメソッドで、tensorの形を変えられる。
y

tensor([-0.1167, -2.0825, -0.0997,  0.7579,  0.1968,  0.9784,  0.0714,  0.4933,
        -1.5637, -0.4070, -0.3432, -2.0151, -1.3312, -0.2478,  0.6550, -1.4580])

In [50]:
z = x.view(-1, 8) #２行８列
z

tensor([[-0.1167, -2.0825, -0.0997,  0.7579,  0.1968,  0.9784,  0.0714,  0.4933],
        [-1.5637, -0.4070, -0.3432, -2.0151, -1.3312, -0.2478,  0.6550, -1.4580]])

 ## Numpyとの接続